In [1]:
import pandas as pd
import numpy as np


df = pd.read_excel(r'C:\Users\faiza\Desktop\web-scraping-\final.xlsx')

In [2]:
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim_models
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

C:\Users\faiza\AppData\Local\Programs\Python\Python39\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
#import nltk
#nltk.download('all')

In [4]:
df.head()

,topic,title,article,date,link
0,BUSINESS,"MAHB faces double whammy of interstate ban, Tu...",KUALA LUMPUR: Affin Hwang Capital expects Mala...,"May 4, 2021 @ 10:40am",https://www.nst.com.my/business/2021/05/687692...
1,BUSINESS,RM opens at 4.09 against USD,KUALA LUMPUR: The ringgit opened easier agains...,"May 3, 2021 @ 10:09am",https://www.nst.com.my/business/2021/05/687398...
2,COLUMNISTS,Biden needs fresh thinking and more than one term,FORGET about the first 100 days. US President ...,"May 3, 2021 @ 12:10am",https://www.nst.com.my/opinion/columnists/2021...
3,SUNDAY VIBES,MONEY THOUGHTS: Ikigai — The uncommon key to c...,HAVE you noticed how life's getting tougher on...,"May 2, 2021 @ 8:30am",https://www.nst.com.my/lifestyle/sunday-vibes/...
4,BUSINESS,"Labuan FSA appoints new chairman, director gen...",KUALA LUMPUR: Labuan Financial Services Author...,"Apr 30, 2021 @ 4:15pm",https://www.nst.com.my/business/2021/04/686731...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   topic    107 non-null    object
 1   title    107 non-null    object
 2   article  107 non-null    object
 3   date     107 non-null    object
 4   link     107 non-null    object
dtypes: object(5)
memory usage: 4.3+ KB


In [6]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [7]:
df['article_clean']=df['article'].apply(clean)

In [8]:
dictionary = corpora.Dictionary(df['article_clean'])
print(dictionary.num_nnz)

20580


In [9]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['article_clean'] ]
print(len(doc_term_matrix))

107


In [10]:
lda = gensim.models.ldamodel.LdaModel

In [32]:
num_topics= 14
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)

Wall time: 5.62 s


In [33]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.017*"said" + 0.014*"police" + 0.012*"investigation" + 0.008*"bank" + 0.007*"public" + 0.006*"coin" + 0.006*"suspect" + 0.006*"code" + 0.006*"section" + 0.006*"cryptocurrency"'),
 (1,
  '0.012*"said" + 0.011*"luno" + 0.009*"malaysia" + 0.008*"cent" + 0.008*"per" + 0.007*"cryptocurrencies" + 0.007*"cryptocurrency" + 0.006*"global" + 0.006*"growth" + 0.005*"2020"'),
 (2,
  '0.019*"said" + 0.012*"per" + 0.011*"cent" + 0.009*"malaysia" + 0.008*"would" + 0.007*"food" + 0.007*"market" + 0.007*"year" + 0.006*"new" + 0.006*"investor"'),
 (3,
  '0.017*"bank" + 0.015*"said" + 0.009*"policy" + 0.009*"sapura" + 0.008*"energy" + 0.006*"housing" + 0.006*"financial" + 0.006*"market" + 0.006*"the" + 0.006*"rate"'),
 (4,
  '0.009*"asset" + 0.008*"said" + 0.006*"economic" + 0.006*"datuk" + 0.005*"malaysia" + 0.004*"wealth" + 0.004*"individual" + 0.004*"time" + 0.004*"currency" + 0.004*"bitcoin"'),
 (5,
  '0.007*"million" + 0.007*"said" + 0.006*"education" + 0.005*"year" + 0.005*"malaysia" + 0.0

In [34]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

In [20]:
lda_corpus = ldamodel[doc_term_matrix]

In [21]:
[doc for doc in lda_corpus]

[[(0, 0.00044462335),
  (1, 0.00044461968),
  (2, 0.00044469043),
  (3, 0.00044462047),
  (4, 0.00044461072),
  (5, 0.00044458974),
  (6, 0.0004446133),
  (7, 0.99599844),
  (8, 0.00044460862),
  (9, 0.00044456788)],
 [(0, 0.0007094711),
  (1, 0.0007094922),
  (2, 0.0007094692),
  (3, 0.99361485),
  (4, 0.00070944463),
  (5, 0.0007094308),
  (6, 0.00070944394),
  (7, 0.0007094942),
  (8, 0.00070945994),
  (9, 0.00070939824)],
 [(0, 0.9990284),
  (1, 0.00010795543),
  (2, 0.000107956555),
  (3, 0.00010797834),
  (4, 0.00010795327),
  (5, 0.00010794808),
  (6, 0.0001079565),
  (7, 0.00010795277),
  (8, 0.000107954),
  (9, 0.00010794582)],
 [(0, 0.9983769),
  (1, 0.00018033732),
  (2, 0.00018034366),
  (3, 0.00018034026),
  (4, 0.00018035306),
  (5, 0.00018032166),
  (6, 0.00018034299),
  (7, 0.00018033272),
  (8, 0.00018033518),
  (9, 0.00018032423)],
 [(0, 0.0007097855),
  (1, 0.0007097601),
  (2, 0.00070981693),
  (3, 0.9936125),
  (4, 0.00070966553),
  (5, 0.0007097022),
  (6, 0.00070

In [22]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

0.10000000000805455
